# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2] #-2 to remove trailing comma

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
import pandas as pd
import numpy as np

### 1. 10 countries with the lowest infant mortality rates

In [7]:
# print names of all countries and their infant mortality rate if it is present
im = []
for country in document.findall( 'country' ):
    if country.find('infant_mortality') != None:
        im.append((country.find('name').text, float(country.find('infant_mortality').text)))
im_df = pd.DataFrame(im,columns=['Country', 'IMRate'])
im_df.sort('IMRate').head(10)

,Country,IMRate
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### 2. 10 cities with the largest population

In [8]:
cpop = []
for country in document.findall( 'country' ):
    if  country.find('province') != None:
        for province in country.iterfind('province'):
            for city in province.iterfind('city'):
                if city.find('population') != None:
                    for pop in city.iterfind('population'):
                        cpop.append((country.find('name').text,city.find('name').text,int(pop.attrib['year']), int(pop.text)))
        
    elif  country.find('city') != None:
        for city in country.iterfind('city'):
            if city.find('population') != None:
                for pop in city.iterfind('population'):
                    cpop.append((country.find('name').text,city.find('name').text,int(pop.attrib['year']), int(pop.text)))
                                
                
cpop_df = pd.DataFrame(cpop,columns=['Country', 'City','Year','Population'])
maxdf = cpop_df.groupby(['Country','City'],as_index = False)['Year'].max()

result  = pd.merge(maxdf, cpop_df,how ='left' ,on=['Country', 'City','Year'] )
result.sort('Population',ascending=False).head(10)
# result[result['City'].str.contains('New York')]
# result[result['City'].str.contains('London')]
# result[result['Country'].str.contains('Uganda')]



,Country,City,Year,Population
644,China,Shanghai,2010,22315474
2522,Turkey,Istanbul,2012,13710512
1215,India,Mumbai,2011,12442373
2113,Russia,Moskva,2013,11979529
469,China,Beijing,2010,11716620
356,Brazil,São Paulo,2010,11152344
672,China,Tianjin,2010,11090314
518,China,Guangzhou,2010,11071424
1184,India,Delhi,2011,11034555
655,China,Shenzhen,2010,10358381


### 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [9]:
# Get populations at country level
CtryPops = []
for country in document.findall( 'country' ):
    if country.find('population') != None:
        CtryPops.append((country.find('name').text, int(country.find('population').text)))
        

CtryPopsdf = pd.DataFrame(CtryPops,columns = ['Country','Population'])

# CtryPopsdf.head()
# CtryPopsdf[CtryPopsdf['Country']== 'Mauritius']

In [10]:
egdata = []
for country in document.findall( 'country' ):
    for eg in country.iterfind('ethnicgroup'):
        egdata.append((country.find('name').text,  float(eg.attrib['percentage']),eg.text ))                              

egdf = pd.DataFrame(egdata,columns=['Country', 'PopPctinCountry','EthnicGrp'])
egdf.head()
egPopdf  = pd.merge(egdf, CtryPopsdf,how ='left' ,on=['Country'] )
egPopdf['EthnicPop'] = egPopdf['PopPctinCountry']/100.0*egPopdf['Population']
result2 = egPopdf.ix[:,['EthnicGrp','EthnicPop']].groupby('EthnicGrp').aggregate(np.sum).sort('EthnicPop',ascending=False)
result2.head(10)

,EthnicPop
EthnicGrp,
Han Chinese,4.975551e+08
European,1.928658e+08
Indo-Aryan,1.716454e+08
Russian,9.275844e+07
African,8.632937e+07
Japanese,8.170627e+07
German,6.623219e+07
Dravidian,5.959908e+07
English,4.231499e+07


### 4. name and country of: 

In [11]:
#create mapping for country abreviations
ctryCodes = {}
for country in document.findall( 'country' ):
    ctryCodes[country.attrib['car_code']] = country.find('name').text

ctryCodes

def getCountryNames(X):
    decodedCountry = ''
    s = X.split()
    for a in s:
        decodedCountry =  decodedCountry + ctryCodes[a] + ', '
    return decodedCountry[:-2].strip()

# getCountryNames('ANG CO')
# ctryCodesdf =pd.DataFrame(ctryCodes,columns = ['Country','CountryCode'])
# ctryCodesdf.head()

####  a) longest river

In [12]:
rivers = []
for river in document.findall( 'river' ):
    if river.find('length') != None:
        rivers.append((river.attrib['country'],  river.find('name').text, float(river.find('length').text)))

riversdf = pd.DataFrame(rivers,columns=['CountryCode','name','length'])
riversdf['Country'] = riversdf['CountryCode'].apply(getCountryNames)

riverresult = riversdf.sort('length',ascending = False).head(1)

riverresult.ix[:,['name','Country']]

,name,Country
174,Amazonas,"Colombia, Brazil, Peru"


####  b) largest lake

In [13]:
lakes=[]
for lake in document.findall( 'lake' ):
    if lake.find('area') != None: #and lake.find('depth') != None:
        lakes.append((lake.attrib['country'],  lake.find('name').text , float(lake.find('area').text))) #, lake.find('depth').text
        
lakesdf = pd.DataFrame(lakes,columns=['CountryCode','name','area'])
lakesdf['Country'] = lakesdf['CountryCode'].apply(getCountryNames)

lakeresult = lakesdf.sort('area',ascending = False).head(1)

lakeresult.ix[:,['name','Country']]        
        

,name,Country
54,Caspian Sea,"Russia, Azerbaijan, Kazakhstan, Iran, Turkmeni..."


####   and c) airport at highest elevation

In [14]:
airports = []
for airport in document.findall( 'airport' ):
    if airport.find('elevation') != None:
        if airport.find('elevation').text != None:
            airports.append((airport.attrib['country'],  airport.attrib['iatacode'],airport.find('name').text, float(airport.find('elevation').text)))

airportsdf = pd.DataFrame(airports,columns=['CountryCode','AirportCode','name','elev'])
airportsdf['Country'] = airportsdf['CountryCode'].apply(getCountryNames)

airportresult = airportsdf.sort('elev',ascending = False).head(1)
airportresult.head()

airportresult.ix[:,['name','Country']]        
  


,name,Country
80,El Alto Intl,Bolivia
